In [1]:
import sys
sys.path.append('..')
import time

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from utils.oanda_utils import get_candles_df, load_candles_df
from utils.indicators import add_sma
from utils.preprocesses import split_df, add_target

In [5]:
from utils.stats import jarque_bera_normal_dist, ljung_box_auto_corr, adfuller_unit_root

In [6]:
import datetime

In [7]:
df = load_candles_df("../data/usd_jpy_m1.csv")
df['time']  = pd.to_datetime(df['time'], utc=True)
df = df.set_index('time')

In [8]:
target_window = 10
df = add_target(df, target_window, align="C")

In [10]:
jarque_bera_normal_dist(df["C"].apply(np.log).diff().dropna())

Jack Bera: 	 Jarque_beraResult(statistic=1815.2714852294166, pvalue=0.0)


In [25]:
ljung_box_auto_corr(df["C"].apply(np.log).diff().dropna(), 10)

1 3.829953173471047 0.05034433924495086
2 4.001606866149845 0.1352265940619419
3 6.374832464841147 0.09473168186660956
4 6.562251344096615 0.16091006226638804
5 6.635558552341858 0.2491852900103888
6 13.177848543679179 0.040297043218276166
7 14.463741159420264 0.04352187517534447
8 16.375998841355077 0.03730390422716841
9 16.52585046231573 0.05667998382022813
10 17.459401315340447 0.06479683329312833


In [17]:
adfuller_unit_root(df["C"].dropna())

---ctt---
(-72.02851048742228, 0.0, 0, 5467, {'1%': -4.373250863916808, '5%': -3.833470663723131, '10%': -3.553929575690367}, -36370.89624698518)
---ct---
(-72.02975968183344, 0.0, 0, 5467, {'1%': -3.960426905874412, '5%': -3.411293375588838, '10%': -3.1275230782317216}, -36372.265174120395)
---c---
(-72.00893646758799, 0.0, 0, 5467, {'1%': -3.4315467025937836, '5%': -2.8620688230848064, '10%': -2.5670514914596914}, -36371.92824350352)
---nc---
(-71.98865314043223, 0.0, 0, 5467, {'1%': -2.566149084220808, '5%': -1.9410492435461104, '10%': -1.6167715082466638}, -36372.10417476253)


In [26]:
df['return_rate'] = df['C'].apply(np.log).diff()

In [45]:
lag_window = 10
features = ["O", "H", "L", "C"]

X = np.zeros([df.shape[0] - lag_window - target_window, lag_window, len(features)])
y = np.zeros([df.shape[0] - lag_window - target_window])

for i in range(len(df))[lag_window:-target_window]:
    for j, f in enumerate(features):
        X[i-lag_window-target_window, :, j] = df[f].iloc[i-lag_window:i]
        y[i-lag_window-target_window] = df['target'].iloc[i]

In [47]:
idx = df.index[lag_window:-target_window]

DatetimeIndex(['2021-03-23 00:10:00+00:00', '2021-03-23 00:11:00+00:00',
               '2021-03-23 00:12:00+00:00', '2021-03-23 00:13:00+00:00',
               '2021-03-23 00:14:00+00:00', '2021-03-23 00:15:00+00:00',
               '2021-03-23 00:16:00+00:00', '2021-03-23 00:17:00+00:00',
               '2021-03-23 00:18:00+00:00', '2021-03-23 00:19:00+00:00',
               ...
               '2021-03-26 20:40:00+00:00', '2021-03-26 20:41:00+00:00',
               '2021-03-26 20:42:00+00:00', '2021-03-26 20:43:00+00:00',
               '2021-03-26 20:44:00+00:00', '2021-03-26 20:45:00+00:00',
               '2021-03-26 20:46:00+00:00', '2021-03-26 20:47:00+00:00',
               '2021-03-26 20:48:00+00:00', '2021-03-26 20:49:00+00:00'],
              dtype='datetime64[ns, UTC]', name='time', length=5449, freq=None)

In [66]:
train_start = datetime.datetime(2021, 3, 23, tzinfo=datetime.timezone.utc)
valid_start = datetime.datetime(2021, 3, 25, tzinfo=datetime.timezone.utc)
test_start = datetime.datetime(2021, 3, 26, tzinfo=datetime.timezone.utc)
test_end = datetime.datetime(2021, 3, 27, tzinfo=datetime.timezone.utc)

In [69]:
df_train, df_valid, df_test = split_df(df.iloc[:-target_window][cols], train_start, valid_start, test_start, test_end)

In [61]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

ModuleNotFoundError: No module named 'keras'

In [ ]:
def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()
 
    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
 
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [ ]:


# ランダムシードの設定
np.random.seed(202)
 
# 初期モデルの構築
yen_model = build_model(train_lstm_in, output_size=1, neurons = 20)
 
# データを流してフィッティングさせましょう
yen_history = yen_model.fit(train_lstm_in, lstm_train_out, 
                            epochs=10, batch_size=1, verbose=2, shuffle=True)
